# Spanish Court Tribunals Webscrape

This script scrapes data from on Spanish Court Tribunals from [this](http://www.minhafp.gob.es/es-ES/Areas%20Tematicas/Contratacion/TACRC/Paginas/BuscadordeResoluciones.aspx?pagina=1) website, before putting the results in a table and exporting to CSV.

## Load in require libraries

In [6]:
import requests #For sending GET requests to websites
from lxml import html #For parsing HTML
import pandas as pd #For creating and manipulating dataframes
import re #For handling regular expressions in text
from datetime import datetime #For processing dates
import timeit #For timing executed code

## Create required functions

In [7]:
#if one string (x) is found within another string (z), replace it with a final string (y)
def clean_issue(x,y,z):
    try:
        output = re.sub(x,y,z)
    
    except TypeError:
        output = ''
    
    return output

#If one string is present in a second string, remove the first string from the second string
def remove_text(x,y):
    try:
        output = re.sub(re.escape(x),'',y)
    
    except TypeError:
        output = None
    
    return output

#Shorten outcome type down to certain number of predefined strings based on a longer string's content
def outcome_reduction(long_outcome):
    
    long_outcome = str(long_outcome)
    
    outcome_list = ['Inadmision y desestimacion','Estimacion parcial',
                    'Desestimacion','Inadmision','Estimacion',
                    'Desistimiento del recurrente'] 

    for item in outcome_list:
        if item in long_outcome:
            long_outcome = item
            break
        else:
            continue

    return(long_outcome)

#If one string is present in a second string, remove the first string from the second string - NOT REQUIRED
def extra_outcome_text(string1,string2):
    try:
        diff_string = re.sub(re.escape(string1),'',string2)
    except TypeError:
        diff_string = None
        
    return(diff_string)

#Split up the legislation column by either a full stop or comma as a seperator
def split_legis(string):
    try:
        snipped = re.split('\\.|,',string)[1]
    except IndexError:
        snipped = ''
    return(snipped)

#Get outcomes from text column
def get_outcome(string1):
    try:    
        string2 = string1.split('.')[1]
    except IndexError:
        string2 = 'NaN'
    return(string2)

#Remove accents from words
def remove_accents(my_string):
    my_string = re.sub('á','a',my_string)
    my_string = re.sub('é','e',my_string)
    my_string = re.sub('í','i',my_string)  
    my_string = re.sub('ó','o',my_string)
    my_string = re.sub('º','o',my_string)
    my_string = re.sub('ú','u',my_string)
    return(my_string)

#Shorten outcome down to certain number of predefined strings based on a longer string's content
def id_outcome_type(outcome_string):
    global contract_type
    if 'inadmision y desestimacion' in outcome_string:
        contract_type = 'Inadmision y Desestimacion'
    elif 'estimacion parcial' in outcome_string:
        contract_type = 'Estimacion Parcial'
    elif 'desestimacion' in outcome_string:
        contract_type = 'Desestimacion'
    elif 'inadmision' in outcome_string:
        contract_type = 'Inadmision'
    elif 'estimacion' in outcome_string:
        contract_type = 'Estimacion'
    elif 'desestimiento del recurrente' in outcome_string:
        contract_type = 'Desestimiento del Recurrente'
    else:
        contract_type = 'NaN'
    return(contract_type)

#Shorten contract type down to certain number of predefined strings based on a longer string's content
def id_contract_type(issue_string):
    global contract_type
    if 'gestion de servicios publicos' in issue_string or 'gestion de servicio publico' in issue_string:
        contract_type = 'Gestion de servicios publicos'
    elif ('servicios' in issue_string and 'suministros' in issue_string) and 'gestion de servicios publicos' not in issue_string:
        contract_type = 'Servicios y suministro'
    elif ('servicio' in issue_string or 'servicios' in issue_string) and 'gestion de servicios publicos' not in issue_string:
        contract_type = 'Servicios'
    elif 'gestion de servicios publicos' in issue_string:
        contract_type = 'gestion de servicios publicos'
    elif 'suministro' in issue_string:
        contract_type = 'Suministro'
    elif 'obras' in issue_string:
        contract_type = 'Obras'
    elif 'concesion de obra publica' in issue_string:
        contract_type = 'Concesion de obra publica'
    elif 'otros' in issue_string:
        contract_type = 'Otros'
    else:
        contract_type = 'NaN'

    return(contract_type)

#Shorten issue down to certain number of predefined strings based on a longer string's content
def id_issue_type(issue_string):
    global issue_type
    if ('Adjudicacion' in issue_string or 'adjudicacion' in issue_string) and ('Exclusion' in issue_string or 'exclusion') in issue_string:
        issue_type = 'Adjudicacion y exclusion'
    if ('Pliegos' in issue_string or 'pliegos' in issue_string) and ('Adjudicacion' in issue_string or 'adjudicacion') in issue_string:
        issue_type = 'Pliegos y adjudicacion'
    if ('Pliegos' in issue_string or 'pliegos' in issue_string) and ('Anuncio' in issue_string or 'anuncio') in issue_string:
        issue_type = 'Pliegos y anuncio'
    if 'Adjudicacion' in issue_string or 'adjudicacion' in issue_string:
        issue_type = 'Adjudicacion'
    elif 'Pliegos' in issue_string or 'pliegos' in issue_string or 'lospliegos' in issue_string or 'PPT' in issue_string or 'PCAP' in issue_string:
        issue_type = 'Pliegos'
    elif 'Criterios de valoracion' in issue_string or 'criterios de valoracion' in issue_string:
        issue_type = 'Criterios de valoracion'
    elif 'Licitacion' in issue_string or 'licitacion' in issue_string:
        issue_type = 'Licitacion'
    elif 'anuncio de convocatoria' in issue_string:
        issue_type = 'Anuncio de convocatoria'
    elif 'acto de apertura de sobres' in issue_string:
        issue_type = 'Acto de apertura de sobres'
    elif 'Exclusion' in issue_string or 'exclusion' in issue_string:
        issue_type = 'Exclusion'
    elif 'Desistimiento' in issue_string or 'Desierta' in issue_string or 'Desierto' in issue_string or 'desistimiento' in issue_string or 'desierta' in issue_string or 'desierto' in issue_string:
        issue_type = 'Desistimiento, Desierta, Desierto'
    elif 'Desistimiento' in issue_string or 'Desierta' in issue_string or 'Desierto' in issue_string or 'desistimiento' in issue_string or 'desierta' in issue_string or 'desierto' in issue_string:
        issue_type = 'Desistimiento, Desierta, Desierto'
    else:
        issue_type = 'NaN'

    return(issue_type)    

#Create a boolean column for if the type of issue is Acuerdo Marco or not
def acuerdo_marco(issue_string):
    if 'Acuerdo Marco' in issue_string or 'AM' in issue_string or 'Acuerdo marco' in issue_string:
        issue_type = True
    else:
        issue_type = False

    return(issue_type)

## Identify total number of pages to be scraped

In [8]:
#URL of first page, which contains total number of pages at bottom
url = 'http://www.minhafp.gob.es/es-ES/Areas%20Tematicas/Contratacion/TACRC/Paginas/BuscadordeResoluciones.aspx?pagina=1'

#Send HTTP request to page
page = requests.get(url)

#Parse page response as HTML tree
tree = html.fromstring(page.content)

#Select XPath for number of pages
no_pages = tree.xpath('//*[@id="ctl00_SPWebPartManager1_g_93bc4c3a_0f69_4097_bed1_978c8b545335"]/div/div/div/p[1]/text()')

#Strip out non-required characters and turn into integer
no_pages = int(re.sub(".*\\/","",no_pages[0]))

#Display
no_pages

655

## Run scrape

In [9]:
#Create empty lists to append each page's data into
resolution_number_all = []
size_all = []
date_all = []
issue_all = []
legislation_all = []
text_all = []

#Set a timer for when the scrape starts, to estimate completion
start = timeit.default_timer()

#Loop through each page number between 1 and the total number of pages identified above
for i in range(1,no_pages+1):
    
    #URL of page
    url = 'http://www.minhafp.gob.es/es-ES/Areas%20Tematicas/Contratacion/TACRC/Paginas/BuscadordeResoluciones.aspx?pagina='+str(i)
    #Send HTTP request to page
    page = requests.get(url)
    #Parse page response as HTML tree
    tree = html.fromstring(page.content)
    
    #Resolution numbers
    resolution_number = tree.xpath("//div[contains(@class, 'breaker_resultados')]/a/text()")
    resolution_number = [re.sub("\\s+","",item) for item in resolution_number]
    resolution_number_all = resolution_number_all + resolution_number
    
    #Size of file
    size = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[2]/text()")
    size = [re.sub("Kb.*","",item) for item in size]
    size = [re.sub(".*pdf","",item) for item in size]
    size = [re.sub(".*doc","",item) for item in size]
    size = [re.sub("\\s+","",item) for item in size]
    size = [re.sub("-","",item) for item in size]
    size = [re.sub(",",".",item) for item in size]
    size = [float(item) for item in size]
    size_all = size_all + size
    
    #Date of resolution
    date = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[4]/text()[2]")
    date = [re.sub("\\s+","",item) for item in date]
    date = [re.sub(":","",item) for item in date]
    date = [datetime.strptime(item, '%d/%m/%Y') for item in date]
    date_all = date_all + date
    
    #Issue raised by complainant
    issue = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[6]/text()[2]")
    issue = [re.sub("\n"," ",item) for item in issue]
    issue = [re.sub(",.*","",item) for item in issue]
    issue = [re.sub("\\s\\s+","",item) for item in issue]
    issue = [re.sub(":","",item) for item in issue]
    issue_all = issue_all + issue
    
    #Legislation applied to test case
    legislation = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[6]/text()[2]")
    legislation = [re.sub(":\r\\s+"," ",item) for item in legislation]
    legislation = [re.sub("\n"," ",item) for item in legislation]
    legislation = [split_legis(item) for item in legislation] 
    legislation = [re.sub("\\s","",item) for item in legislation]
    legislation_all = legislation_all + legislation
    
    #Free text associated with resolution
    text = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[6]/text()[2]")
    #text = [re.sub("\n"," ",item) for item in text]
    text_all = text_all + text
    
    #Record time
    stop = timeit.default_timer()
    print(str(i),
          '| Time Elapsed',
          str(round((stop - start)/60,2)),
         'mins | Estimated Completion: ' + str(round((stop - start)/i * no_pages/60,2)),
         'mins')

#Convert to a dataframe
tribunals_df = pd.DataFrame({'resolution_number':resolution_number_all,
                             'size': size_all,
                             'date': date_all,
                             'issue': issue_all,
                             'legislation': legislation_all,
                             'text': text_all})

#Select certain columns
tribunals_df = tribunals_df[['date', 'issue', 'legislation','resolution_number', 'size', 'text']]

1 | Time Elapsed 0.01 mins | Estimated Completion: 7.87 mins
2 | Time Elapsed 0.02 mins | Estimated Completion: 7.32 mins
3 | Time Elapsed 0.03 mins | Estimated Completion: 7.09 mins
4 | Time Elapsed 0.04 mins | Estimated Completion: 7.01 mins
5 | Time Elapsed 0.06 mins | Estimated Completion: 7.63 mins
6 | Time Elapsed 0.07 mins | Estimated Completion: 7.4 mins
7 | Time Elapsed 0.08 mins | Estimated Completion: 7.36 mins
8 | Time Elapsed 0.09 mins | Estimated Completion: 7.18 mins
9 | Time Elapsed 0.09 mins | Estimated Completion: 6.87 mins
10 | Time Elapsed 0.1 mins | Estimated Completion: 6.83 mins
11 | Time Elapsed 0.11 mins | Estimated Completion: 6.82 mins
12 | Time Elapsed 0.12 mins | Estimated Completion: 6.81 mins
13 | Time Elapsed 0.13 mins | Estimated Completion: 6.8 mins
14 | Time Elapsed 0.15 mins | Estimated Completion: 6.79 mins
15 | Time Elapsed 0.16 mins | Estimated Completion: 6.78 mins
16 | Time Elapsed 0.17 mins | Estimated Completion: 6.78 mins
17 | Time Elapsed 0.

134 | Time Elapsed 1.29 mins | Estimated Completion: 6.32 mins
135 | Time Elapsed 1.3 mins | Estimated Completion: 6.32 mins
136 | Time Elapsed 1.31 mins | Estimated Completion: 6.32 mins
137 | Time Elapsed 1.32 mins | Estimated Completion: 6.31 mins
138 | Time Elapsed 1.33 mins | Estimated Completion: 6.3 mins
139 | Time Elapsed 1.34 mins | Estimated Completion: 6.3 mins
140 | Time Elapsed 1.34 mins | Estimated Completion: 6.29 mins
141 | Time Elapsed 1.35 mins | Estimated Completion: 6.28 mins
142 | Time Elapsed 1.36 mins | Estimated Completion: 6.27 mins
143 | Time Elapsed 1.37 mins | Estimated Completion: 6.26 mins
144 | Time Elapsed 1.38 mins | Estimated Completion: 6.25 mins
145 | Time Elapsed 1.38 mins | Estimated Completion: 6.25 mins
146 | Time Elapsed 1.39 mins | Estimated Completion: 6.24 mins
147 | Time Elapsed 1.4 mins | Estimated Completion: 6.24 mins
148 | Time Elapsed 1.41 mins | Estimated Completion: 6.24 mins
149 | Time Elapsed 1.42 mins | Estimated Completion: 6.25 m

265 | Time Elapsed 2.57 mins | Estimated Completion: 6.36 mins
266 | Time Elapsed 2.58 mins | Estimated Completion: 6.36 mins
267 | Time Elapsed 2.59 mins | Estimated Completion: 6.36 mins
268 | Time Elapsed 2.6 mins | Estimated Completion: 6.36 mins
269 | Time Elapsed 2.61 mins | Estimated Completion: 6.36 mins
270 | Time Elapsed 2.62 mins | Estimated Completion: 6.36 mins
271 | Time Elapsed 2.63 mins | Estimated Completion: 6.35 mins
272 | Time Elapsed 2.63 mins | Estimated Completion: 6.34 mins
273 | Time Elapsed 2.64 mins | Estimated Completion: 6.34 mins
274 | Time Elapsed 2.65 mins | Estimated Completion: 6.34 mins
275 | Time Elapsed 2.66 mins | Estimated Completion: 6.34 mins
276 | Time Elapsed 2.67 mins | Estimated Completion: 6.35 mins
277 | Time Elapsed 2.68 mins | Estimated Completion: 6.34 mins
278 | Time Elapsed 2.69 mins | Estimated Completion: 6.33 mins
279 | Time Elapsed 2.7 mins | Estimated Completion: 6.34 mins
280 | Time Elapsed 2.71 mins | Estimated Completion: 6.33

396 | Time Elapsed 3.78 mins | Estimated Completion: 6.25 mins
397 | Time Elapsed 3.79 mins | Estimated Completion: 6.25 mins
398 | Time Elapsed 3.8 mins | Estimated Completion: 6.25 mins
399 | Time Elapsed 3.81 mins | Estimated Completion: 6.25 mins
400 | Time Elapsed 3.82 mins | Estimated Completion: 6.25 mins
401 | Time Elapsed 3.83 mins | Estimated Completion: 6.25 mins
402 | Time Elapsed 3.84 mins | Estimated Completion: 6.25 mins
403 | Time Elapsed 3.85 mins | Estimated Completion: 6.26 mins
404 | Time Elapsed 3.86 mins | Estimated Completion: 6.26 mins
405 | Time Elapsed 3.87 mins | Estimated Completion: 6.26 mins
406 | Time Elapsed 3.88 mins | Estimated Completion: 6.26 mins
407 | Time Elapsed 3.89 mins | Estimated Completion: 6.26 mins
408 | Time Elapsed 3.9 mins | Estimated Completion: 6.27 mins
409 | Time Elapsed 3.91 mins | Estimated Completion: 6.26 mins
410 | Time Elapsed 3.92 mins | Estimated Completion: 6.27 mins
411 | Time Elapsed 3.93 mins | Estimated Completion: 6.27

527 | Time Elapsed 4.97 mins | Estimated Completion: 6.18 mins
528 | Time Elapsed 4.98 mins | Estimated Completion: 6.18 mins
529 | Time Elapsed 4.99 mins | Estimated Completion: 6.18 mins
530 | Time Elapsed 5.0 mins | Estimated Completion: 6.18 mins
531 | Time Elapsed 5.01 mins | Estimated Completion: 6.18 mins
532 | Time Elapsed 5.02 mins | Estimated Completion: 6.18 mins
533 | Time Elapsed 5.02 mins | Estimated Completion: 6.17 mins
534 | Time Elapsed 5.03 mins | Estimated Completion: 6.17 mins
535 | Time Elapsed 5.04 mins | Estimated Completion: 6.17 mins
536 | Time Elapsed 5.05 mins | Estimated Completion: 6.17 mins
537 | Time Elapsed 5.06 mins | Estimated Completion: 6.17 mins
538 | Time Elapsed 5.06 mins | Estimated Completion: 6.16 mins
539 | Time Elapsed 5.07 mins | Estimated Completion: 6.16 mins
540 | Time Elapsed 5.08 mins | Estimated Completion: 6.16 mins
541 | Time Elapsed 5.09 mins | Estimated Completion: 6.16 mins
542 | Time Elapsed 5.09 mins | Estimated Completion: 6.1

## Clean data

In [10]:
#Clean issue column
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: clean_issue('\r','',x))
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: clean_issue('\n','',x))

#Clean text column
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub(':\r\n\\s+','',x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\r','',x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\\s\\s+','',x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\n','',x))

#Remove issue text from free text column
tribunals_df['text'] = tribunals_df.apply(lambda df: remove_text(df['issue'],df['text']),axis=1)

#Limit legislation to a number of categories
tribunals_df['legislation'] = pd.Categorical(tribunals_df['legislation'], 
                                             categories=['TRLCSP','LCSE','LCSPDS','UTE','TRCLSP','TRLCS','TRLCPS',
                                                        'TRLSCP','LCSP','SA','LCSEP','PN','RLCSP','SA(SEGITTUR)',
                                                        'TRLCSPE','TRLKCSP','SA(TRAGSA)','TRLCSPDS','LCSPD',
                                                        'LCSSPDS'], ordered=False)

#Remove legislation text from free text column to form outcome of the resolution
tribunals_df['outcome'] = tribunals_df.apply(lambda df: remove_text(df['legislation'],df['text']),axis=1)

#Clean outcome column
tribunals_df['outcome'] = tribunals_df['outcome'].map(lambda x: re.sub('\\s\\.',' . ',str(x)))
tribunals_df['outcome'] = tribunals_df['outcome'].map(lambda x: re.sub('^,\\s\\.\\s','',x))
tribunals_df['outcome'] = tribunals_df['outcome'].map(lambda x: re.sub('\\..*','',x))

#Clean text column
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\\.','. ',x))

#Create temporary second outcome column which reduces outcome to a number of categories
tribunals_df['outcome2'] = tribunals_df['outcome'].map(lambda x: outcome_reduction(x))

#Remove strings not required from outcome column
tribunals_df['outcome_text'] = tribunals_df.apply(lambda df: remove_text(df['outcome2'],df['outcome']),axis=1)

#Any blank outcomes are set to NaN
tribunals_df.loc[tribunals_df['outcome'].str.len() == 0, 'outcome'] = "NaN"

#Clean up issue column
tribunals_df['text2'] = tribunals_df['issue'].map(lambda x: str(x))
tribunals_df['text2'] = tribunals_df['text2'].map(lambda x: re.sub('\\.[0-9]\\.-',', ',x))
tribunals_df['text2'] = tribunals_df['text2'].map(lambda x: re.sub('[0-9]\\.-','',x))
tribunals_df['issue'] = tribunals_df['text2'].map(lambda x: re.sub('\\..*','',x))
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: re.sub('^\\s','',x))

#Select first item in outcome column to clean up
tribunals_df['outcome2'] = tribunals_df['text'].map(lambda x: get_outcome(x))

#Remove accents
tribunals_df['outcome2'] = tribunals_df['outcome2'].map(lambda x: remove_accents(x))

#Correct spelling mistake
tribunals_df['outcome2'][tribunals_df['outcome2'] == 'Inadmisisón'] = 'Inadmisión'

#Set any cells in temporary outcome column which are NaN to NaN in permanent column
tribunals_df['outcome'][tribunals_df['outcome'] == 'NaN'] = tribunals_df['outcome2'][tribunals_df['outcome'] == 'NaN']

#Turn temporary outcome column to lowercase for ease of manipulation
tribunals_df['outcome2'] = tribunals_df['outcome2'].str.lower()

#Clean outcomes
tribunals_df['outcome2'] = tribunals_df['outcome2'].map(lambda x: id_outcome_type(x))

tribunals_df['outcome'] = pd.Categorical(tribunals_df['outcome2'], 
                                             categories=['Inadmision y Desestimacion','Estimacion Parcial',
                                                            'Desestimacion','Inadmision','Estimacion',
                                                        'Desistimiento del Recurrente', 'NaN'],
                                          ordered=False)

#Clean text column further, including removing any text already in legislation or outcome columns
tribunals_df['text'] = tribunals_df.apply(lambda df: remove_text(df['legislation'],df['text']),axis=1)
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('^,\\s\\.\\s','',str(x)))
tribunals_df['text'] = tribunals_df.apply(lambda df: remove_text(df['outcome'],df['text']),axis=1)
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('^\\.\\s','',str(x)))

#Set any empty text cells to NaN
tribunals_df.loc[tribunals_df['text'].str.len() == 0, 'text'] = "NaN"

#Remove defunct columns
tribunals_df = tribunals_df[['resolution_number','date','size','issue','legislation','outcome','text']]

#Clean outcome column further, removing superfluous text
tribunals_df['outcome2'] = tribunals_df['outcome'].map(lambda x: outcome_reduction(x))
tribunals_df['outcome_text'] = tribunals_df.apply(lambda df: remove_text(df['outcome2'],df['outcome']),axis=1)

#Remove defunct columns
tribunals_df = tribunals_df[['resolution_number','date','size','issue','legislation','outcome','text']]

#Remove accents
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: remove_accents(x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: remove_accents(x))

#Clean contract type
tribunals_df['contract'] = tribunals_df['issue'].map(lambda x: id_contract_type(x))

#Create Acuerdo Marco column
tribunals_df['acuerdo_marco'] = tribunals_df['issue'].map(lambda x: acuerdo_marco(x))

#Clean issue type column
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: id_issue_type(x))

tribunals_df

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,resolution_number,date,size,issue,legislation,outcome,text,contract,acuerdo_marco
0,0862/2018,2018-10-01,145.89,Pliegos,LCSP,Inadmision,Inadmision. Falta de legitimacion del recurr...,Servicios,False
1,0848/2018,2018-10-01,246.45,Exclusion,LCSP,Estimacion,Estimacion. No se considera que la indicacio...,Servicios,False
2,0854/2018,2018-10-01,121.17,NaN,LCSP,Desestimacion,Desestimacion. Impugnacion de anuncio. Legi...,Gestion de servicios publicos,False
3,855./2018,2018-10-01,306.97,Adjudicacion,LCSP,Desestimacion,Desestimacion. Impugnacion indirecta de los ...,Servicios,False
4,0880/2018,2018-10-01,332.41,Adjudicacion,LCSP,Desestimacion,Desestimacion. Se considera por la recurrent...,Servicios,False
5,0852/2018,2018-10-01,156.15,Exclusion,LCSP,Inadmision,Inadmision. Recurso presentado fuera de plazo.,Servicios,False
6,0872/2018,2018-10-01,175.44,"Desistimiento, Desierta, Desierto",LCSP,Inadmision,Inadmision. Falta sobrevenida de objeto del ...,Servicios,False
7,0851/2018,2018-10-01,133.22,Pliegos,LCSP,Estimacion,Estimacion. Incumplimiento por el pliego del...,Suministro,False
8,0849/2018,2018-10-01,129.16,Adjudicacion,LCSP,Desestimacion,Desestimacion. Se fundamenta en defecto de m...,Servicios,False
9,0853/2018,2018-10-01,86.80,Pliegos,LCSP,Inadmision,Inadmision. Falta de identificacion de activ...,Servicios,False


## Output to CSV

In [11]:
#Output to CSV
tribunals_df.to_csv("../output/tribunals2.csv", encoding="utf-8")